In [1]:
using PyCall
using Distances
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using Distances
using DataFrames
# using PyPlot
using SparseArrays
using Printf
using Images

In [2]:
py"""
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
import cv2
import os

class SceneReconstruction3D:

    def __init__(self,K, dist):
        self.K = K
        self.K_inv = np.linalg.inv(K)
        self.d = dist

    def load_image_pair(self, img_path1, img_path2, use_pyr_down=True):
        self.img1 = cv2.imread(img_path1, cv2.CV_8UC3)
        self.img2 = cv2.imread(img_path2, cv2.CV_8UC3)

        if self.img1 is None:
            sys.exit("Image " + img_path1 + " could not be loaded.")
        if self.img2 is None:
            sys.exit("Image " + img_path2 + " could not be loaded.")

        if len(self.img1.shape) == 2:
            self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_GRAY2BGR)
            self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_GRAY2BGR)

        target_width = 200
        if use_pyr_down and self.img1.shape[1] > target_width:
            while self.img1.shape[1] > 2 * target_width:
                self.img1 = cv2.pyrDown(self.img1)
                self.img2 = cv2.pyrDown(self.img2)

        self.img1 = cv2.undistort(self.img1, self.K, self.d)
        self.img2 = cv2.undistort(self.img2, self.K, self.d)

    def findRootSIFTFeatures(self):
        class RootSIFT:
            def __init__(self):
                self.extractor = cv2.xfeatures2d.SIFT_create()

            def compute(self, image, kps, eps=1e-7):
                (kps, descs) = self.extractor.compute(image, kps)
                if len(kps) == 0:
                    return ([], None)

                descs /= (descs.sum(axis=1, keepdims=True) + eps)
                descs = np.sqrt(descs)
                return (kps, descs)

        class InnerFeatures:
            def __init__(self, kps, des, pos):
                self.kps = kps
                self.des = des
                self.pos = pos

        def innerRootSIFT(img):
            sift = cv2.xfeatures2d.SIFT_create()
            (kps, descs) = sift.detectAndCompute(img, None)

            rs = RootSIFT()
            (kps, descs) = rs.compute(img, kps)
            pos = [np.array([x.pt[0], x.pt[1]]) for x in kps]

            return kps, descs, pos

        kps1, desc1, pos1 = innerRootSIFT(self.img1)
        kps2, desc2, pos2 = innerRootSIFT(self.img2)
        self.feature_1 = InnerFeatures(kps1, desc1, pos1)
        self.feature_2 = InnerFeatures(kps2, desc2, pos2)

K = np.array([[2759.48/4, 0, 1520.69/4, 0, 2764.16/4,1006.81/4, 0, 0, 1]]).reshape(3, 3)
d = np.array([0.0, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5)
scene = SceneReconstruction3D(K,d)

"""

In [3]:
img1_path = "../data/pair/Left.png"
img2_path = "../data/pair/Right.png"

py"scene.load_image_pair"(img1_path, img2_path)
py"scene.findRootSIFTFeatures()"

pts1 = py"scene.feature_1.pos"
pts2 = py"scene.feature_2.pos";

__main__:48: RuntimeWarning: invalid value encountered in sqrt


In [6]:
function cleaning(original)
    res_dict = Dict()
    for i in 1:size(original, 1)
        res_dict[hash(original[i,:])] = original[i,:]
    end
    
    vals = collect(values(res_dict))
    output = zeros(size(vals,1), 2)
    for i in 1:size(vals,1)
        output[i,:] = [vals[i][1][1],vals[i][1][2]]
    end
    
    return output
end

P_points = cleaning(pts1)
Q_points = cleaning(pts2)

println("size P points", size(P_points))
println("size Q points", size(Q_points))

size P points(326, 2)
size Q points(358, 2)


In [7]:
cost = pairwise(Euclidean(), P_points, Q_points; dims=1)
println(size(cost))
P = ones(size(P_points,1))
Q = ones(size(Q_points,1));

(326, 358)


In [8]:
solCount = 10
# m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0,PoolGap = 0.001))
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(P), sum(Q)))
@constraint(m, X * ones(Int, length(Q)) .<= P)
@constraint(m, X'ones(Int, length(P)) .<= Q);
optimize!(m)
solution_pool = zeros(solCount, length(P),length(Q))
cnt = 0
obj = objective_value(m)

for i in 0:(solCount-1)
    try
        setparam!(m.moi_backend.inner,"SolutionNumber", i)
        xn = Gurobi.get_dblattrarray(m.moi_backend.inner, "Xn", 1, length(X))
        xn_val = Gurobi.get_dblattr(m.moi_backend.inner, "PoolObjVal")
        if(round(xn_val,digits=1) != round(obj, digits=1))
            println(cnt , " solution(s) selected")
            break
        end
        default = zeros(length(P),length(Q))
        for i in 0:length(P)-1
            default[i+1,:] = xn[(i*length(Q))+1:(i+1)*length(Q)]
        end
        solution_pool[i+1,:,:] = default
        cnt+=1
    catch 
        break
    end
end
sol_pool = deepcopy(solution_pool[1:cnt,:,:]);

Optimize a model with 685 rows, 116708 columns and 350124 nonzeros
Variable types: 0 continuous, 116708 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+02]
Presolve time: 0.35s
Presolved: 685 rows, 116708 columns, 350124 nonzeros
Variable types: 0 continuous, 116708 integer (116708 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      1s
       1       1370   1.0003113e+07   2.9791175e+02      1s
       2       3397   3.8509861e+03   1.1551795e+03      1s
       3       4685   3.7452621e+03   3.1133642e+03      1s

Sifting complete


Root relaxation: objective 3.730867e+03, 5453 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [12]:
solOther = sparse(sol_pool[1,:,:])
sizeOf = min(size(P,1), size(Q,1))
matched_pts1 = zeros(sizeOf,2)
matched_pts2 = zeros(sizeOf,2)
i = 1

for (x,y,v) in zip(findnz(solOther)...)
    x_pos = [P_points'[:,x][1], Q_points'[:,y][1]]
    y_pos = [P_points'[:,x][2], Q_points'[:,y][2]]
    matched_pts1[i,:] = [floor(x_pos[1]) floor(y_pos[1])]
    matched_pts2[i,:] = [floor(x_pos[2]) floor(y_pos[2])]
    i+=1
end

In [13]:
df = DataFrame()
df.PX = matched_pts1[:,1]
df.PY = matched_pts1[:,2]
df.QX = matched_pts2[:,1]
df.QY = matched_pts2[:,2];
df

,PX,PY,QX,QY
,Float64,Float64,Float64,Float64
1,58.0,122.0,46.0,141.0
2,354.0,130.0,377.0,135.0
3,351.0,32.0,355.0,35.0
4,100.0,149.0,99.0,148.0
5,216.0,122.0,219.0,122.0
6,280.0,97.0,286.0,99.0
7,216.0,100.0,208.0,100.0
8,120.0,221.0,97.0,212.0
9,227.0,94.0,220.0,94.0


In [14]:
CSV.write("../data/pair/matchedPoints.csv",  df, writeheader=false)

"../data/pair/matchedPoints.csv"